In [2]:
# Import standard library modules
import sys

# Set the relative path to the project root directory
relative_path_to_root = "../../"

# Add the project root to the system path for importing in-house modules
sys.path.append(relative_path_to_root)

# Import in-house modules from the 'utilities' package
from utilities import clean_historical_data, check_tickers_for_missing_values
from utilities import calc_vif, calc_p_values, calc_correlation, highlight_vif, highlight_p_values, evaluate_regression_model, evaluate_cross_validation, evaluate_classifier_model
from utilities import load_data, save_data

In [3]:
# Data manipulation and analysis
import pandas as pd

# Date and time manipulation
from datetime import date

# File and directory manipulation
from pathlib import Path

# Data preprocessing and model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [5]:
file_name = "todays_stuff"
file_path = f"../../data/raw_data/{file_name}"

raw_data = load_data(file_path)

╔═══════════════════════════════════════════════════════════════╗
║      File `test2.csv.bz2` loaded from `todays_stuff.zip`      ║
╚═══════════════════════════════════════════════════════════════╝


In [6]:
raw_data

,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
0,131.619995,233.490005,189.740005,135.820007,114.129997,105.650002,361.450012,481.170013,230.139999,57.240002,...,292.119995,31.969999,98.610001,64.510002,118.959999,130.835007,134.869995,103.550003,363.529999,182.850006


In [9]:
# Pivot columns: (tickers), to rows in a new column called tickers
# Make the values in a new column called Adjusted Close

pivot_data = raw_data.melt(var_name="Tickers", value_name="Adjusted Close")

pivot_data


,Tickers,Adjusted Close
0,A,131.619995
1,AAPL,233.490005
2,ABBV,189.740005
3,ABNB,135.820007
4,ABT,114.129997
...,...,...
496,XYL,130.835007
497,YUM,134.869995
498,ZBH,103.550003
499,ZBRA,363.529999


In [ ]:
# Load model from file

import pickle
# Import the LogisticRegression class from the linear_model module

model_name = "clf_XGB_v1.pkl"
model_path = f"../../models/{model_name}"

with open(model_path, 'rb') as file:
    loaded_model = pickle.load(file)

# Use the loaded model to make predictions
predictions = loaded_model.predict(pivot_data)
